# **Carlos Gross-Martinez**

#**Assignmnet 2 - Elastic Net**

#**CAP6640 - Computational Foundations of AI**



***Importing libraries to use in the program and importing the information from the excel sheet to create a data frame for data manipulation and processing. Then, updating values from qualitative to quatitave for features that need to be updated. Finally, printing the dataframe to verify information.***

In [ ]:
import pandas as pd
import numpy as np

data_import = pd.read_csv("Credit_N400_p9.csv")
updated_data_import = data_import.replace({'Gender':{'Male': 1, 'Female': 0}, 'Student':{'Yes': 1, 'No': 0}, 'Married':{'Yes': 1, 'No': 0}}) 

updated_data_import

***Converting pd dataframe into np array for manipulation. Next, centering and standarizing the features. Then, centering the output. Finally, creating X and y matrices for elastic net training and printing results to screen to verify information***

In [ ]:
data_set = updated_data_import.to_numpy()

y = 0
X = []

for feature in range(data_set.shape[1]):

  if feature == data_set.shape[1] - 1:
    y = np.subtract(data_set[:, feature], np.mean(data_set[:, feature]))
  else:
    X.append(np.divide(np.subtract(data_set[:, feature], np.mean(data_set[:, feature])), np.std(data_set[:, feature])))

X = np.asarray(X)
X = X.T

print(type(y))
print(y.shape)
print(y)
print(type(X))
print(X.shape)
print(X)

***Defining the class elastic regression that will conduct elastic net using coordinate descent in the dataset. It will loop through different tunning parameters and return the beta values for all tunning parameters in an array.***

In [ ]:
class ElasticRegression() :
    def __init__( self, learning_rate, iterations, tuningParam1, tuningParam2) :
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.tuningParam1 = tuningParam1
        self.tuningParam2 = tuningParam2

    def fit( self, X, Y ) :
        self.exampleNum, self.ftnum = X.shape
        self.W = np.random.uniform(-1, 1, 9)
        self.b = 0 
        self.X = X 
        self.Y = Y 

        for i in range( self.iterations ) :   
            self.update_weights()
        return self
      
    def update_weights( self ) :
        Y_pred = self.predict( self.X )
        dW = np.random.uniform(-1, 1, 9)
        for j in range( self.ftnum ) : 
            if self.W[j] > 0 :  
                dW[j] = ( - ( 2 * ( self.X[:,j] ).dot( self.Y - Y_pred ) ) +          
                         self.tuningParam1 + 2 * self.tuningParam2 * self.W[j] ) / self.exampleNum
            else :
                dW[j] = ( - ( 2 * ( self.X[:,j] ).dot( self.Y - Y_pred ) )      
                         - self.tuningParam1 + 2 * self.tuningParam2 * self.W[j] ) / self.exampleNum
        db = - 2 * np.sum( self.Y - Y_pred ) / self.exampleNum
        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db
        return self

    def predict( self, X ) :
        return X.dot( self.W ) + self.b

***Driver program to obtain coeficients from elastic net training. It will loop through all tunning parameters (alphas and lamdas) to calaculate best beta values***

In [ ]:
#9 Features plot of coefficients.
Tuning_parameters = [10**-2,10**-1,10**0,10**1,10**2,10**3,10**4, 10**5, 10**6]
Tuning_parameters2 = [0,1/5,2/5,3/5,4/5,1]
lr = 10**-5
iteration = 1000

coef = []

for tuningParameter in Tuning_parameters:
  nineParams = []
  for tuningParameter2 in Tuning_parameters2:
    elastic = ElasticRegression(lr,iteration,tuningParameter2,tuningParameter)
    elastic.fit(X,y)
    nineParams.append(elastic.W)
  coef.append(nineParams)
print(coef)

# ***Deliverable 1***

***Plotting all 6 graphs with the lambdas for each alpha***

In [ ]:
#Deliverable 1
import matplotlib.pyplot as plt
pltfeatures = ['Income','Limit','Rating','Cards','Age','Education','Gender','Student','Married']

for x in range(0,6):
  Income = []
  Limit = []
  Rating = []
  Cards = []
  Age = []
  Education = []
  Gender = []
  Student = []
  Married = []
  for i in range(0,9):
    Income.append(coef[i][x][0])
    Limit.append(coef[i][x][1])
    Rating.append(coef[i][x][2])
    Cards.append(coef[i][x][3])
    Age.append(coef[i][x][4])
    Education.append(coef[i][x][5])
    Gender.append(coef[i][x][6])
    Student.append(coef[i][x][7])
    Married.append(coef[i][x][8])
  fig, ax = plt.subplots()
  plt.xscale('log')
  plt.title(str(Tuning_parameters2[x]) +" tuning parameter")
  ax.plot(Tuning_parameters,Income,label="Income")
  ax.plot(Tuning_parameters,Limit,label="Limit")
  ax.plot(Tuning_parameters,Rating,label="Rating")
  ax.plot(Tuning_parameters,Cards,label="Cards")
  ax.plot(Tuning_parameters,Age,label="Age")
  ax.plot(Tuning_parameters,Education,label="Education")
  ax.plot(Tuning_parameters,Gender,label="Gender")
  ax.plot(Tuning_parameters,Student,label="Student")
  ax.plot(Tuning_parameters,Married,label="Married")
  ax.legend(loc = 'upper left')
  plt.show()

***Defining a function to split a datasets into folds. The function takes the datasets X and y as well as a fold number which will split the data set and return the partitioned dataset***

In [ ]:
from random import randrange

def cross_val_split(x,y, folds):
	features = []
	labels = []
	x_copy = list(x)
	y_copy = list(y)
	fold_size = int(len(x) / folds)
	for i in range(folds):
		x_fold = []
		y_fold = []
		while len(x_fold) < fold_size:
			index = randrange(len(x_copy))
			x_fold.append(x_copy.pop(index))
			y_fold.append(y_copy.pop(index))
		features.append(x_fold)
		labels.append(y_fold)
	return features,labels

***Training model with cross validations and calculating MSE results for plotting***

In [ ]:
#Training the model using Cross Validation
Tuning_parameters = [10**-2,10**-1,10**0,10**1,10**2,10**3,10**4, 10**5, 10**6]
Tuning_parameters2 = [0,1/5,2/5,3/5,4/5,1]
lr = 10**-5
iteration = 1000
fold_num = 5

ft,lb= cross_val_split(X,y,5)

finalResults = []
for tuningParameter2 in Tuning_parameters2:
  mseResults = []
  for tuningParameter in Tuning_parameters:
    model = ElasticRegression(lr,iteration,tuningParameter2,tuningParameter)
    foldMSE = []
    for i in range(0,fold_num):

      valx = ft[i]
      valy = lb[i]
      trainx = []
      trainy = []
      for x in range (0,fold_num):
        if x != i:
          trainx += ft[x]
          trainy += lb[x]
      trainx = np.array(trainx)
      trainy = np.array(trainy)
      valx = np.array(valx)
      valy = np.array(valy)

      model.fit(trainx,trainy)
      prediction = model.predict(valx)
      n = len(valy)

      squaredError = ((valy - prediction) ** 2)
      error = np.mean(squaredError)

      foldMSE.append(error)

    meanFoldMSE = np.mean(foldMSE)
    mseResults.append(meanFoldMSE)
  finalResults.append(mseResults)
print(finalResults)

# ***Deliverable 2***

***Plotting all MSe values obtained from cross validation based on each alpha value***

In [ ]:
#Deliverable 2
import math
import matplotlib.pyplot as plt
count = 0
for mse in finalResults:
  fig, ax = plt.subplots()
  plt.xscale('log')
  plt.plot(Tuning_parameters, mse)
  ax.set_ylabel("MSE")
  ax.set_xlabel('\u03BB')
  ax.set_title( str(count) +' alpha 5 CV Tuning Parameters')
  plt.show()
  count += 0.2
  count = round(count, 1)

# ***Deliverable 3***

***Obtaining the pair of values for lambda and alpha that provides the lowest MSE error***

In [ ]:
mins = []
minindexes = []
mintuns = []
mintuns2 = []
for mse in finalResults:
  minCVError = min(mse)
  minCVIndex = mse.index(minCVError)
  𝜆 = Tuning_parameters[minCVIndex]
  α = Tuning_parameters2[minCVIndex]
  mins.append(minCVError)
  mintuns.append(𝜆)
  mintuns2.append(α)

finalMin = min(mins)
minα = mins.index(finalMin)

for mse in finalResults:
  if finalMin in mse:
    min𝜆 = mse.index(finalMin)

print("Optimal α: ", Tuning_parameters2[minα])
print("Optimal 𝜆: ", Tuning_parameters[min𝜆])
print("Minimum CV(5) error: ", finalResults[minα][min𝜆])

# ***Deliverable 4***

***Retraining the model with the lamda and alpha values provided the lowest MSE, and printing the beta values obtained from the traning***

In [ ]:
optimalLamda = min𝜆
optimalAlpha = minα
lr = 10**-5
iteration = 10000
coef = []

elastic = ElasticRegression(lr,iteration,optimalLamda,optimalAlpha)
elastic.fit(X,y)
coef = elastic.W
print("Best Beta Estimate:", coef)